# Data Wrangling
[Data wrangeling](https://en.wikipedia.org/wiki/Data_wrangling) is the process of transforming and mapping data from a "raw" format into a format more valuable for further downstream pruposes such as analytics. Read more about data. 

Data wrangling can be divided into teo steps

1. Data acquisition
2. Data cleaning


## Data acquisition

Some ways too aquire data can be
    
* Downloading files
* Accessing an API
* Scraping a web page
* Combine data from different formats

### Comma Separated Values (CSV)
 A [comma separated value (CSV)](https://en.wikipedia.org/wiki/Comma-separated_values) file is a delimited text file that uses comma to seppate values. The CSV is easy to process with code (unlike [.xlsx](https://fileinfo.com/extension/xlsx)). In Python the contents of a CSV file are commonly represented as a list of . There are two common choices for how to represent each row. In the first option, each row is a list.

In [1]:
# Option 1: Each row is a list
csv = [['Q', 'W', 'E'],
       ['R', 'T', 'Y']]

In the second option each row is a dictionary. This option works well if you have a CSV header because then the keys of each dictionary can be column names and the fields can be values. 

In [2]:
# Option 2: Each row is a dictionary
csv = [{'name1': 'Q', 'name2': 'W', 'name3': 'E'},
       {'name1': 'R', 'name2': 'T', 'name3': 'Y'}]

## Loading data from CSVs
### Python's csv Module
We will be using the `unicodecsv` since it comes with Anaconda and has support for unicode. The `unicodecsv` works exactly the same as Python's [`csv`](https://docs.python.org/2/library/csv.html) module, and its documentation page is still the best way to learn how to use the `unicodecsv` library.
### Data file
Lets have a look at our data file before reading it with `unicodecsv`.

In [3]:
file_extract = ''
with open('data_files/enrollments.csv', 'r') as f:
    for index, line in enumerate(f):
        file_extract += line
        if index == 5:
            break
print(file_extract)

account_key,status,join_date,cancel_date,days_to_cancel,is_udacity,is_canceled
448,canceled,2014-11-10,2015-01-14,65,True,True
448,canceled,2014-11-05,2014-11-10,5,True,True
448,canceled,2015-01-27,2015-01-27,0,True,True
448,canceled,2014-11-10,2014-11-10,0,True,True
448,current,2015-03-10,,,True,False



### Loading the data

Next we will load the data from some file using `unicodecsv`. The mode `rb` in `open('...', 'rb')` means that the file will be opened for reading. The [`csv`](https://docs.python.org/2/library/csv.html) docummentation page mentions that we need to use this. `rb` stands for Read Binary mode. We are using the `DictReader` since our data have a header row. Our reader will be an iterator, the difference between lists and iteratiors in Python can be found [here](https://www.codementor.io/sheena/python-generators-and-iterators-du1082iua). The iterator let's you write a loop to access each element, but only once. 

In [4]:
import unicodecsv
from pprint import pprint

enrollments = []

with open('data_files/enrollments.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    enrollments = list(reader)

pprint(enrollments[0])

OrderedDict([('account_key', '448'),
             ('status', 'canceled'),
             ('join_date', '2014-11-10'),
             ('cancel_date', '2015-01-14'),
             ('days_to_cancel', '65'),
             ('is_udacity', 'True'),
             ('is_canceled', 'True')])


Let's read in two more example files.

In [5]:
daily_engagement = []
project_submsissions = []

def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

daily_engagement = read_csv('data_files/daily_engagement.csv')
project_submissions = read_csv('data_files/project_submissions.csv')

pprint(daily_engagement[0])
pprint(project_submissions[0])

OrderedDict([('acct', '0'),
             ('utc_date', '2015-01-09'),
             ('num_courses_visited', '1.0'),
             ('total_minutes_visited', '11.6793745'),
             ('lessons_completed', '0.0'),
             ('projects_completed', '0.0')])
OrderedDict([('creation_date', '2015-01-14'),
             ('completion_date', '2015-01-16'),
             ('assigned_rating', 'UNGRADED'),
             ('account_key', '256'),
             ('lesson_key', '3176718735'),
             ('processing_state', 'EVALUATED')])


## Fixing Data Types
All the data that we have read in is represented as strings, it's up to us to convert them to the correct data types. 

In [6]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)
# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
pprint(enrollments[0])

OrderedDict([('account_key', '448'),
             ('status', 'canceled'),
             ('join_date', datetime.datetime(2014, 11, 10, 0, 0)),
             ('cancel_date', datetime.datetime(2015, 1, 14, 0, 0)),
             ('days_to_cancel', 65),
             ('is_udacity', True),
             ('is_canceled', True)])


In [7]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
pprint(daily_engagement[0])

OrderedDict([('acct', '0'),
             ('utc_date', datetime.datetime(2015, 1, 9, 0, 0)),
             ('num_courses_visited', 1),
             ('total_minutes_visited', 11.6793745),
             ('lessons_completed', 0),
             ('projects_completed', 0)])


In [8]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])
pprint(project_submissions[0])

OrderedDict([('creation_date', datetime.datetime(2015, 1, 14, 0, 0)),
             ('completion_date', datetime.datetime(2015, 1, 16, 0, 0)),
             ('assigned_rating', 'UNGRADED'),
             ('account_key', '256'),
             ('lesson_key', '3176718735'),
             ('processing_state', 'EVALUATED')])


## Investigating the Data
Let's find the number of rows and number of unique students in each of the csv files.

In [9]:
enrollments_num_of_rows = len(enrollments)
engagements_num_of_rows_daily = len(daily_engagement)
submissions_num_of_rows_project = len(project_submissions)

enrollments_num_of_unique_students = len(set([enrollment['account_key'] for enrollment in enrollments]))
daily_engagement_num_of_unique_students = len(set([de['acct'] for de in daily_engagement]))
project_submissions_num_of_unique_students = len(set([ps['account_key'] for ps in project_submissions]))

print(f'enrollments; number of rows: {enrollments_num_of_rows}, number of unique students: {enrollments_num_of_unique_students}')
print(f'daily_engagement; number of rows: {engagements_num_of_rows_daily}, number of unique students: {daily_engagement_num_of_unique_students}')
print(f'project_submsissions; number of rows: {submissions_num_of_rows_project}, number of unique students: {project_submissions_num_of_unique_students}')



enrollments; number of rows: 1640, number of unique students: 1302
daily_engagement; number of rows: 136240, number of unique students: 1237
project_submsissions; number of rows: 3642, number of unique students: 743


### Problems in the Data


There are a couple of problems in the data. Fistly, there are more students in enrollment than engagement table, there should have been the same amount in both  tables. Secondly, a column is named 'account_key' in two of the tables and 'acct' in the third, this really isen't a big problem, but it is inconvenient. We will start with changing 'acct' to 'account_key'.

In [10]:
## Rename the "acct" column in the daily_engagement table to "account_key".

for engagement in daily_engagement:
    engagement['account_key'] = engagement['acct']
    del engagement['acct']
daily_engagement
print(daily_engagement[0]['account_key'])

0


We can now make a function for all the data sets to get out unique students,

In [11]:
def get_unique_students(data):
    unique_students = set()
    for data_point in data:
        unique_students.add(data_point['account_key'])
    return unique_students

unique_enrolled_students = get_unique_students(enrollments)
unique_engagement_students = get_unique_students(daily_engagement)
unique_project_submitters = get_unique_students(project_submissions)


Now on to the second problem, why are students missing from `daily_engagement`? Here is a process to investigate the problem:
1. Identify suprising datapoints
    - Any enrollment record with no corresponding engagement data
2. Print out one or a few surprising data points
3. Fix any problems you find
    - More investigation may be necessary
    - Or there might not be a problem!

In [12]:
missing_students = list(unique_enrolled_students - unique_engagement_students)
for student in enrollments:
    if student['account_key'] in missing_students and student['days_to_cancel'] != 0:
        pprint(student)
    

OrderedDict([('account_key', '1304'),
             ('status', 'canceled'),
             ('join_date', datetime.datetime(2015, 1, 10, 0, 0)),
             ('cancel_date', datetime.datetime(2015, 3, 10, 0, 0)),
             ('days_to_cancel', 59),
             ('is_udacity', True),
             ('is_canceled', True)])
OrderedDict([('account_key', '1304'),
             ('status', 'canceled'),
             ('join_date', datetime.datetime(2015, 3, 10, 0, 0)),
             ('cancel_date', datetime.datetime(2015, 6, 17, 0, 0)),
             ('days_to_cancel', 99),
             ('is_udacity', True),
             ('is_canceled', True)])
OrderedDict([('account_key', '1101'),
             ('status', 'current'),
             ('join_date', datetime.datetime(2015, 2, 25, 0, 0)),
             ('cancel_date', None),
             ('days_to_cancel', None),
             ('is_udacity', True),
             ('is_canceled', False)])


We discover that the majority of the missing students are students who cancled their subscription the same day they enlisted. Three of the missing students who divirge from this pattern are Udacity test accounts, and we don't want them in our analysis data set. We will remove all Udacity test accounts from our data set.

In [13]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
print(len(udacity_test_accounts))

6


In [14]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [15]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_engagement))
print(len(non_udacity_submissions))

1622
135656
3634
